In [1]:
import glob
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
filename = '/home/<user>/melvin/xml/0001172661-19-002217.txt'

In [4]:
with open(filename) as fp:
    text = fp.read()
    secHeader = text[text.find('<SEC-HEADER>'):text.find('</SEC-HEADER>')]

In [7]:
fp = open('/home/<user>/melvin/xml/0001172661-19-002217.txt')
soup = BeautifulSoup(fp, 'lxml')

In [47]:
results = {
    'report_date': [],
    'submission_type': [],
    'nameofissuer': [],
    'cusip': [],
    'value': [],
    'sshprnamt': [],
    'sshprnamttype': [],
    'putcall': [],  
}

for subDir in glob.glob('/home/<user>/melvin/xml/*.txt'):
    fp = open(subDir)
    soup = BeautifulSoup(fp, 'lxml')
    tags = list(soup.find_all('document'))
        
    report_date = tags[0].find('reportcalendarorquarter').text
    submission_type = tags[0].find('submissiontype').text
    
    for x in tags[1].find_all('infotable'):
        results['report_date'].append(report_date)
        results['submission_type'].append(submission_type)
        results['nameofissuer'].append(x.nameofissuer.text)
        results['cusip'].append(x.cusip.text)
        results['value'].append(x.value.text)
        results['sshprnamt'].append(x.shrsorprnamt.sshprnamt.text)
        results['sshprnamttype'].append(x.shrsorprnamt.sshprnamttype.text)
        putcall = None 
        if x.putcall is not None:
            putcall = x.putcall.text 
        results['putcall'].append(putcall)
    
    if len(tags)>2:
        print(subDir)
    

In [22]:
tags = list(soup.find_all('document'))

In [26]:
report_date = tags[0].find('reportcalendarorquarter').text

In [41]:
tags[0].find('submissiontype').text

'13F-HR'

In [30]:
all_holding = list(tags[1].find_all('infotable'))

In [31]:
len(all_holding)

105

In [36]:
all_holding[0]

<infotable>
<nameofissuer>ADOBE INC</nameofissuer>
<titleofclass>COM</titleofclass>
<cusip>00724F101</cusip>
<value>146413</value>
<shrsorprnamt>
<sshprnamt>530000</sshprnamt>
<sshprnamttype>SH</sshprnamttype>
</shrsorprnamt>
<putcall>Call</putcall>
<investmentdiscretion>SOLE</investmentdiscretion>
<votingauthority>
<sole>530000</sole>
<shared>0</shared>
<none>0</none>
</votingauthority>
</infotable>

In [39]:

for x in tags[1].find_all('infotable'):
    results['report_date'].append(report_date)
    results['submission_type'].append(submission_type)
    results['nameofissuer'].append(x.nameofissuer.text)
    results['cusip'].append(x.cusip.text)
    results['value'].append(x.value.text)
    results['sshprnamt'].append(x.shrsorprnamt.sshprnamt.text)
    results['sshprnamttype'].append(x.shrsorprnamt.sshprnamttype.text)
    putcall = None 
    if x.putcall is not None:
        putcall = x.putcall.text 
    results['putcall'].append(putcall)

In [49]:
df = pd.DataFrame(results)

In [50]:
df.to_csv('melvin_holding.csv', index=False)

In [51]:
only_shares = df.loc[df['putcall'].isnull(),:]

In [52]:
only_shares.head()

,report_date,submission_type,nameofissuer,cusip,value,sshprnamt,sshprnamttype,putcall
1,12-31-2016,13F-HR,ADOBE SYS INC,00724F101,72065,700000,SH,None
2,12-31-2016,13F-HR,ADVANCE AUTO PARTS INC,00751Y106,93016,550000,SH,None
3,12-31-2016,13F-HR,ALPHABET INC,02079K305,174339,220000,SH,None
7,12-31-2016,13F-HR,AVON PRODS INC,054303102,18144,3600000,SH,None
9,12-31-2016,13F-HR,BRUNSWICK CORP,117043109,46359,850000,SH,None


In [53]:
haha = only_shares.groupby('nameofissuer').size()

In [55]:
haha[haha==1]

nameofissuer
ACV AUCTIONS INC            1
AFFIRM HLDGS INC            1
AIRBNB INC                  1
ALASKA AIR GROUP INC        1
ALTICE USA INC              1
                           ..
WARNER MUSIC GROUP CORP     1
WEIBO CORP                  1
WHIRLPOOL CORP              1
WHOLE FOODS MKT INC         1
WYNDHAM DESTINATIONS INC    1
Length: 83, dtype: int64

In [57]:
grouped = only_shares.groupby("nameofissuer").agg(
    dates=("report_date", "first"),
    count=("report_date", "size"),
)

In [59]:
grouped.loc[grouped['count']==1, :].tail(20)

,dates,count
nameofissuer,,
SLACK TECHNOLOGIES INC,06-30-2019,1
SMILEDIRECTCLUB INC,03-31-2020,1
SMITH & WESSON HLDG CORP,12-31-2015,1
SNAP ON INC,12-31-2016,1
STITCH FIX INC,03-31-2018,1
STRYKER CORP,09-30-2018,1
TAPESTRY INC,03-31-2021,1
TIFFANY & CO NEW,03-31-2015,1
TUPPERWARE BRANDS CORP,09-30-2020,1


In [16]:
soup.find_all('document')

[<document>
 <type>13F-HR
 <sequence>1
 <filename>primary_doc.xml
 <text>
 <xml>
 <?xml version="1.0" encoding="UTF-8"?>
 <edgarsubmission xmlns="http://www.sec.gov/edgar/thirteenffiler" xmlns:ns1="http://www.sec.gov/edgar/common" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemalocation="http://www.sec.gov/edgar/thirteenffiler eis_13F_Filer.xsd">
 <headerdata>
 <submissiontype>13F-HR</submissiontype>
 <filerinfo>
 <livetestflag>LIVE</livetestflag>
 <flags>
 <confirmingcopyflag>false</confirmingcopyflag>
 <returncopyflag>true</returncopyflag>
 <overrideinternetflag>false</overrideinternetflag>
 </flags>
 <filer>
 <credentials>
 <cik>0001628110</cik>
 <ccc>XXXXXXXX</ccc>
 </credentials>
 </filer>
 <periodofreport>09-30-2019</periodofreport>
 </filerinfo>
 </headerdata>
 <formdata>
 <coverpage>
 <reportcalendarorquarter>09-30-2019</reportcalendarorquarter>
 <isamendment>false</isamendment>
 <filingmanager>
 <name>Melvin Capital Management LP</name>
 <address>
 <ns1:street1

In [2]:
count = 0
all_er = {'cik':[], 'ErDates': [], 'filename': []}
for subDir in glob.glob('/mnt/harddrive/sec_cik/*/'):
    itemInfo = []
    cik = subDir.split('/')[-2]
    for filename in glob.glob(f'{subDir}8-K/*.txt'):
        #filename='./fb_8k/fb/8-K/0001326801-19-000066.txt'
        hasER = False
        with open(filename) as fp:
            #print(filename)
            #soup = BeautifulSoup(fp, 'html.parser')
            #secHeader = soup.find('sec-header')
            text = fp.read()
            secHeader = text[text.find('<SEC-HEADER>'):text.find('</SEC-HEADER>')]
            headerSplits = secHeader.split('\n')
            for tt in headerSplits:
                if 'ITEM INFORMATION' in tt:
                    itemInfo.append(tt)
                if 'Results of Operations and Financial Condition' in tt:
                    hasER = True
                    #print(secHeader)
                    break
            if hasER:
                for tt in headerSplits:
                    if 'FILED AS OF DATE' in tt:
                        #print(tt)
                        #print(tt.split(':')[1].strip())                    
                        erdates = tt.split(':')[1].strip()
                        all_er['cik'].append(cik)
                        all_er['ErDates'].append(erdates)
                        all_er['filename'].append(filename)
                        #print(f"cik={cik}, er_dates={erdates}")
                        break
    count+=1
    #if count>5:
    #    break

KeyboardInterrupt: 